In [1]:
from tqdm import tnrange
import numpy as np
from Crypto.Cipher import AES

In [2]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    [0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76], # 0
    [0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0], # 1
    [0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15], # 2
    [0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75], # 3
    [0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84], # 4
    [0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf], # 5
    [0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8], # 6
    [0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2], # 7
    [0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73], # 8
    [0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb], # 9
    [0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79], # a
    [0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08], # b
    [0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a], # c
    [0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e], # d
    [0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf], # e
    [0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16]  # f
]

In [3]:
def aes_sbox(inputdata, key):
    input_s = inputdata ^ key
    if input_s < 16:
        return sbox[0][input_s]
    row = input_s // 16
    col =  input_s - (16 * row)
    return sbox[row][col]

In [4]:
def get_bit(output, bit):
    try:
        return int(bin(output)[-(bit + 1)])
    except:
        return 0

In [5]:
def guess_leakage(keyguess, inputdata, bit):
    return get_bit(aes_sbox(keyguess, inputdata), bit)

In [6]:
# settings for generic setup
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'

# run the setup script
%run "helper_scripts/Setup_Generic.ipynb"

ModuleNotFoundError: No module named 'nbformat'

In [ ]:
# flash the target firmware to the ChipWhisperer STM32 chip
target_firmware = "aes.hex"
cw.program_target(scope, prog, target_firmware)

In [ ]:
# find the number of recorded traces and data points per trace from the trace_array

numtraces = 10000 #total number of traces
numpoints = 5000 #samples per trace

In [ ]:
##### # Perform the capture, resulting in trace_array and textin_array of 500 traces.
# set up key text pattern (ktp) acquisition: Basic works just fine we only use the random text.
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

_, text = ktp.next()

N = numtraces # number of traces
for i in tnrange(N, desc='Capturing traces'):
    
    # Arm the ChipWhisperer with scope.arm(). 
    # It will begin capturing as soon as it is triggered (which in our case is a rising edge on gpio4).
    # The gpio4 is set high from the firmware as soon as it receives the message.
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    # scope.capture() will read back the captured power trace, blocking until either ChipWhisperer
    # is done recording, or the scope times out.
    # Note that the error return will tell you whether or not the scope timed out. 
    # It does not return the captured scope data.
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    # You can read back the captured power trace with scope.get_last_trace()
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    _, text = ktp.next() 

In [ ]:
# In guess = Byte guess
# In byte_to_attack = Byteindex
# In bit_to_check = bit position in the leakage
# Out return = list with abs(one_averages - zero_averages) 
"""
Loops through all the traces and take the `bit_to_check` into consideration
"""
def calculate_diffs(guess, byte_to_attack=0, bit_to_check=0):
    """Perform a simple DPA on two traces, uses global `textin_array` and `trace_array` """

    one_list = []
    zero_list = []

    for trace_index in range(numtraces):
        
        input_byte = textin_array[trace_index][byte_to_attack]
            
        #Get a simulated leakage list - use aes_sbox(guess, input_byte)
        simulated_leakage = guess_leakage(guess, input_byte, bit_to_check)

        if simulated_leakage == 1:
            one_list.append(trace_array[trace_index])
        else:
            zero_list.append(trace_array[trace_index])
                
    one_avg = np.asarray(one_list).mean(axis=0)
    zero_avg = np.asarray(zero_list).mean(axis=0)

    return abs(one_avg - zero_avg)

In [ ]:
bit_check = 3
key_guess = []
for byte in tnrange(0, 16, desc=f"Byte {byte} wird gebrochen"):
    mean_diffs = [0] * 256
    for guess in range(256):
        mean_diffs[guess] = np.max(calculate_diffs(guess, byte, bit_check))
    get_highest = np.argsort(mean_diffs)[::-1]
    key_guess.append(hex(get_highest[0]))

print(f"Der berechnete Schlüssel ist: {key_guess}")  

In [ ]:
key_guess = ['0x48', '0x61', '0x72', '0x64', '0x77', '0x61', '0x72', '0x65', '0x53', '0x65', '0x63', '0x75', '0x72', '0x69', '0x74', '0x79']
msg = b'g\xf4\x9b\xf2\xe7[\xf5)g2V\xcau\x89\xfe\xa1'
key = ""
for d in key_guess:
    key += chr(int(d, 16))
decipher = AES.new(key, AES.MODE_ECB)
print(decipher.decrypt(msg))